# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [35]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic.metrics import metrics

from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.models.als_recommender import ALSRecommender
from sponge_bob_magic.models.knn_recommender import KNNRecommender
from sponge_bob_magic.models.lightfm_recommender import LightFMRecommender

from sponge_bob_magic.scenarios.main_scenario.main_factory import MainScenarioFactory
from sponge_bob_magic.utils import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [10]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [11]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [36]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings

,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


users

,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067


items

,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [37]:
popular_recommender = PopularRecommender(spark)
als_recommender = ALSRecommender(spark)
knn_recommender = KNNRecommender(spark)
lightfm_recommender = LightFMRecommender(spark)

In [38]:
log_bydate_splitter = log_splitter.LogSplitByDateSplitter(
    spark, True, True,
    datetime(2007, 1, 1)
)

log_random_splitter = log_splitter.LogSplitRandomlySplitter(
    spark, True, True,
    test_size=0.3, seed=1234
)

log_cold_splitter = log_splitter.ColdUsersExtractingSplitter(
    spark, True, True,
    test_size=0.3
)

user_random_splitter = user_log_splitter.RandomUserLogSplitter(
    spark, True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

user_bydate_splitter = user_log_splitter.ByTimeUserLogSplitter(
    spark, True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

In [39]:
factory = MainScenarioFactory(spark)

### Обучение сценария <a name="fit-scenario"></a>

In [40]:
results = None

In [44]:
scenario = factory.get(
    splitter=user_random_splitter,
    recommender=lightfm_recommender,
    criterion=metrics.HitRateMetric(spark),
    metrics=[
        metrics.NDCGMetric(spark),
        metrics.PrecisionMetric(spark),
        metrics.MAPMetric(spark),
        metrics.RecallMetric(spark),
        metrics.Surprisal(spark, log),
    ],
    fallback_recommender=popular_recommender,
)

In [45]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}

als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}

lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}

knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [46]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

29-Jan-20 15:06:51, root, DEBUG: Деление лога на обучающую и тестовую выборку


29-Jan-20 15:07:03, root, DEBUG: Длина трейна и теста: (84013, 15944)


29-Jan-20 15:07:04, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490


29-Jan-20 15:07:05, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355


29-Jan-20 15:07:05, root, DEBUG: Обучение и предсказание дополнительной модели


...

29-Jan-20 15:09:01, root, DEBUG: -- Метрики: HitRate@K=0.8429 nDCG@k=0.2407 Precision@k=0.2306 MAP@k=0.0435 Recall@K=0.0978 Surprisal@K=1.6978


[I 2020-01-29 15:09:01,283]

 Finished trial#1 resulted in value: 0.8428571428571429. Current best value is 0.8714285714285714 with parameters: {'rank': 21}.

29-Jan-20 15:09:01, root, DEBUG: Лучшие значения метрики: 0.8714285714285714


29-Jan-20 15:09:01, root, DEBUG: Лучшие параметры: {'rank': 21}


In [47]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

,number,value,datetime_start,datetime_complete,params_rank,user_attrs_MAP@k,user_attrs_Precision@k,user_attrs_Recall@K,user_attrs_Surprisal@K,user_attrs_nDCG@k,system_attrs__number,state
0,0,0.871429,2020-01-29 15:07:14.675498,2020-01-29 15:08:08.757402,21,0.043425,0.236327,0.102137,1.550649,0.251074,0,COMPLETE
1,1,0.842857,2020-01-29 15:08:08.861441,2020-01-29 15:09:01.283284,75,0.043508,0.230612,0.097768,1.697821,0.240673,1,COMPLETE


### Получение рекомендаций <a name="predict-scenario"></a>

In [48]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

29-Jan-20 15:09:01, root, DEBUG: Проверка датафреймов


29-Jan-20 15:09:01, root, DEBUG: Предварительная стадия обучения (pre-fit)


29-Jan-20 15:09:02, root, DEBUG: Основная стадия обучения (fit)


29-Jan-20 15:09:02, root, DEBUG: Построение модели LightFM


29-Jan-20 15:09:03, root, DEBUG: Проверка датафреймов


29-Jan-20 15:09:03, root, DEBUG: Выделение дефолтных юзеров


29-Jan-20 15:09:03, root, DEBUG: Выделение дефолтных айтемов


In [49]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    199|  0.4170581102371216|no_context|
|     91|    174| 0.14767946302890778|no_context|
|     91|    511|0.026487989351153374|no_context|
|     91|    480|-0.02488577924668789|no_context|
|     91|    510|-0.02957684360444...|no_context|
|     91|    435|-0.04762391000986099|no_context|
|     91|    197|-0.06133553013205528|no_context|
|     91|    194|-0.10036946833133698|no_context|
|     91|    187|-0.15194128453731537|no_context|
|     91|    483|-0.16255392134189606|no_context|
|    152|     15| -0.4885908365249634|no_context|
|    152|     88| -0.7593095898628235|no_context|
|    152|    393| -0.7923286557197571|no_context|
|    152|    125| -0.8461843729019165|no_context|
|    152|    111| -0.8694582581520081|no_context|
|    152|    660| -0.9163415431976318|no_context|
|    152|    402|  -0.948605477809906|no_context|
